In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Create a DataFrame representing the rating matrix
data = {
    "Northanger Abby": [5, 1, 1, np.nan],
    "Wuthering Heights": [4, 2, 2, 4],
    "Oroonoko": [3, 4, 3, 3],
    "Bondswoman’s Narrative": [4, 5, np.nan, 1]
}
index = ["Alex", "Loren", "Taylor", "Ainsley"]
df = pd.DataFrame(data, index=index)

In [3]:
# Compute item-item similarity using cosine similarity
item_similarity = cosine_similarity(df.fillna(0).T)
item_similarity_df = pd.DataFrame(item_similarity, index=df.columns, columns=df.columns)

In [4]:
# Function to predict missing ratings
def predict_rating(df, item_similarity_df):
    df_filled = df.copy()
    
    for book in df.columns:
        for user in df.index:
            if pd.isna(df.at[user, book]):  # If rating is missing
                
                # Find similar books and corresponding user's ratings
                sim_books = item_similarity_df[book].drop(book)
                rated_books = df.loc[user].dropna()
                
                # Compute weighted rating
                numerator = sum(sim_books[rated_books.index] * rated_books)
                denominator = sum(abs(sim_books[rated_books.index]))
                
                # Predict rating if denominator is not zero
                if denominator > 0:
                    df_filled.at[user, book] = numerator / denominator
                else:
                    df_filled.at[user, book] = df.mean().mean()  # Default to global mean
    
    return df_filled

In [5]:
# Fill missing ratings
filled_df = predict_rating(df, item_similarity_df)
print("\nFilled Rating Table:\n", filled_df)


Filled Rating Table:
          Northanger Abby  Wuthering Heights  Oroonoko  Bondswoman’s Narrative
Alex            5.000000                  4         3                4.000000
Loren           1.000000                  2         4                5.000000
Taylor          1.000000                  2         3                2.035334
Ainsley         2.643833                  4         3                1.000000
